<a href="https://colab.research.google.com/github/jeongeun/exercise/blob/master/Gen_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
pip install awkward1 mplhep uproot3

     |████████████████████████████████| 117 kB 6.1 MB/s 
     |████████████████████████████████| 14.5 MB 12.3 MB/s 
     |████████████████████████████████| 11.2 MB 21.6 MB/s 
     |████████████████████████████████| 5.8 MB 50.3 MB/s 
     |████████████████████████████████| 884 kB 45.2 MB/s 
     |████████████████████████████████| 87 kB 5.3 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
python -version


NameError: ignored

In [ ]:
import uproot3
import numpy as np
import awkward1 as ak
import matplotlib.pyplot as plt
import mplhep as hep
from uproot3_methods import TVector2Array, TLorentzVectorArray

In [ ]:
dat = uproot3.lazyarrays('drive/MyDrive/WZG/*.root',"Delphes")

In [ ]:
PID_arr = dat['Particle.PID']
Ppt_arr = dat['Particle.PT']
Peta_arr = dat['Particle.Eta']
Pphi_arr = dat['Particle.Phi']
PE_arr = dat['Particle.E']
PC_arr = dat['Particle.Charge']

In [ ]:
elept,eleeta,elephi,eleE,eleC = Ppt_arr[PID_arr == 11], Peta_arr[PID_arr == 11],Pphi_arr[PID_arr == 11],PE_arr[PID_arr == 11],PC_arr[PID_arr == 11]
enupt,enueta,enuphi,enuE = Ppt_arr[PID_arr == -12], Peta_arr[PID_arr == -12],Pphi_arr[PID_arr == -12],PE_arr[PID_arr == -12]

mupt,mueta,muphi,muE,muC = Ppt_arr[PID_arr == 13], Peta_arr[PID_arr == 13],Pphi_arr[PID_arr == 13],PE_arr[PID_arr == 13],PC_arr[PID_arr == 13]
mnupt,mnueta,mnuphi,mnuE = Ppt_arr[PID_arr == -14], Peta_arr[PID_arr == -14],Pphi_arr[PID_arr == -14],PE_arr[PID_arr == -14]

In [ ]:
Electron = ak.zip({

"PT" : elept,
"Eta" : eleeta,
"Phi" : elephi,
"Charge" : eleC,
"T" : eleE

})

Enu = ak.zip({

"PT" : enupt,
"Eta" : enueta,
"Phi" : enuphi,
"T" : enuE

})

Muon = ak.zip({

"PT" : mupt,
"Eta" : mueta,
"Phi" : muphi,
"Charge" : muC,
"T" : muE

})

Mnu = ak.zip({

"PT" : mnupt,
"Eta" : mnueta,
"Phi" : mnuphi,
"T" : mnuE

})

In [ ]:
def PT_cut(Electron, Enu, Muon, Mnu):

        Electron_mask = Electron.PT > 25
        Enu_mask = Enu.PT > 25
        Muon_mask = Muon.PT > 25
        Mnu_mask = Mnu.PT > 25

        return Electron_mask, Enu_mask, Muon_mask, Mnu_mask

In [ ]:
Electron_mask, Enu_mask, Muon_mask, Mnu_mask = PT_cut(Electron, Enu, Muon, Mnu)

In [ ]:
Electron = Electron[Electron_mask]
Enu = Enu[Enu_mask]
Muon = Muon[Muon_mask]
Mnu = Mnu[Mnu_mask]

In [ ]:
Ele_eem_mask = ak.num(Electron) > 1
Mu_eem_mask = ak.num(Muon) > 0
Enu_eem_mask = ak.num(Enu) == 0
Mnu_eem_mask = ak.num(Mnu) > 0

In [ ]:
eem_mask = Ele_eem_mask & Mu_eem_mask & Enu_eem_mask & Mnu_eem_mask

eem_ele = Electron[eem_mask]
eem_mu = Muon[eem_mask]
eem_Enu = Enu[eem_mask]
eem_Mnu = Mnu[eem_mask]

In [ ]:
eem_lead_mu = eem_mu[ak.argmax(eem_mu.PT, axis=1, keepdims=True)]
eem_Mnu = eem_Mnu[ak.argmax(eem_Mnu.PT, axis=1, keepdims=True)]

In [ ]:
muon_pt = eem_lead_mu.PT
MET_MET = eem_Mnu.PT

muon_phi = eem_lead_mu.Phi
MET_phi = eem_Mnu.Phi

muon_pt = ak.flatten(muon_pt)
MET_MET = ak.flatten(MET_MET)

muon_phi = ak.flatten(muon_phi)
MET_phi = ak.flatten(MET_phi)

In [ ]:
print(muon_pt)
print(len(muon_pt))
print(len(MET_MET))

In [ ]:
lep3_T2Vec = TVector2Array.from_polar(muon_pt, muon_phi)
eem_MET_T2Vec = TVector2Array.from_polar(MET_MET, MET_phi)

MT_m = np.sqrt(2*(muon_pt)*(MET_MET)*(1-np.cos(np.abs(eem_MET_T2Vec.delta_phi(lep3_T2Vec)))))

In [ ]:
histo = {}
histo['MT'] = MT_m

outname = "eem_MT.npy"

np.save(outname, histo)